# nii-dg を利用したガバナンス事例

以下では、nii-dg 機能を利用した研究データガバナンスの一例として、 Sapporo によるワークフロー実行結果の検証を行う流れを示す。
Sapporo については [GitHub - sapporo-wes/sapporo-service](https://github.com/sapporo-wes/sapporo-service) を参照のこと。

## 前提

- Sapporo を用いたワークフロー実行の結果を、研究データとしてパッケージングし RO-Crate を生成する
- Sapporo と NII-DG REST API server は、Docker container として起動し、同一ネットワークに接続されている
- 検証として、以下の三点を確認する
  1. RO-Crate 内の値に基づいて、ワークフローが再実行可能である
  2. 再実行のステータスが、RO-Crate に記載されているステータスと同一である
  3. 再実行で得られた結果ファイルのサイズ・チェックサムが、RO-Crate の値と同一である (RO-Crate に記載がある場合のみ)

## 0.1. Sapporo and NII-DG REST API Server の起動

```bash
# At this directory (sapporo_example)
$ docker compose up -d
[+] Running 2/2
 ✔ Container sapporo-service  Running                                                                0.0s
 ✔ Container nii-dg           Started                                                                0.9s
$ docker compose ps
NAME                IMAGE                                       COMMAND                  SERVICE             CREATED              STATUS              PORTS
nii-dg              nii-dg                                      "tini -- python /app…"   nii-dg              12 seconds ago       Up 11 seconds       0.0.0.0:5000->5000/tcp
sapporo-service     ghcr.io/sapporo-wes/sapporo-service:1.4.9   "tini -- uwsgi --yam…"   sapporo-service     About a minute ago   Up About a minute   0.0.0.0:1122->1122/tcp
```

疎通確認として、

```bash
# Host -> Sapporo
$ curl localhost:1122/service-info
{
  "auth_instructions_url"...
}

# Host -> NII-DG REST API Server
$ curl localhost:5000/healthcheck
{"message":"OK"}

# NII-DG REST API Server -> Sapporo
$ docker compose exec nii-dg bash
# In container
$ apt update
$ apt install -y curl
$ curl sapporo-service:1122/service-info
{
  "auth_instructions_url"...
}
```

## 0.2. NII-DG library の install

`0.1. Sapporo and NII-DG REST API Server の起動` にて、起動した NII-DG container は、validation のためのものである。
この項では、packaging 用の nii-dg library を install する。

```bash
# root directory に移動する
$ cd ../
$ ls setup.py
setup.py
$ python3 -m pip install .
...
$ python3 -m pip list | grep nii-dg
nii-dg                 1.0.0
```

## 1. Sapporo でのワークフロー実行 (`execute_workflow.py`)

実行する workflow として、[trimming_and_qc.cwl](https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/trimming_and_qc.cwl) を用いる。
この workflow は CommonWorkflowLanguage (CWL) によって書かれており、FASTQ ファイル (塩基配列ファイル) を入力とし、FASTQC による QC と、Trimmomatic によるトリミングを行う。

workflow を `curl` を用いて実行する場合、以下のようになる。(下のコマンドの実行は skip してよい)

```bash
$ curl -X POST \
     -F 'workflow_params={"fastq_1":{"location":"ERR034597_1.small.fq.gz","class":"File"},"fastq_2":{"location":"ERR034597_2.small.fq.gz","class":"File"},"nthreads":2}' \
     -F 'workflow_type=CWL' \
     -F 'workflow_type_version=v1.0' \
     -F 'workflow_url=https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/trimming_and_qc.cwl' \
     -F 'workflow_engine_name=cwltool' \
     -F 'workflow_attachment=[{"file_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/ERR034597_2.small.fq.gz","file_name": "ERR034597_2.small.fq.gz"},{"file_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/ERR034597_1.small.fq.gz","file_name": "ERR034597_1.small.fq.gz"}]' \
     http://localhost:1122/runs
{'run_id': 'd45a404e-8d8a-43ac-bbf8-b686ee426062'}
```

この command を python で実装したものとして、`execute_workflow.py` であり、これを用いて、ワークフローを実行する。

```bash
$ python3 execute_workflow.py -h
usage: execute_workflow.py [-h] [endpoint]

positional arguments:
  endpoint    The endpoint to send the request (default: http://localhost:1122)

optional arguments:
  -h, --help  show this help message and exit

$ python3 execute_workflow.py http://localhost:1122
{'run_id': 'd45a404e-8d8a-43ac-bbf8-b686ee426062'}
```

Sapporo の `RUN_ID` は、出力される `d45a404e-8d8a-43ac-bbf8-b686ee426062` などである。
実際に Sapporo における run の状態の確認は、この `RUN_ID` を用いて行う。
(run の状態などの元の source は、`${PWD}/run` 以下に配置されている)

```bash
$ curl localhost:1122/runs/d45a404e-8d8a-43ac-bbf8-b686ee426062 | jq .state
"COMPLETE"
```

この state が `COMPLETE` になったら、次のステップに進む。

## 2. Sapporo でのワークフロー実行結果の取得 (`download_results.py`)

想定として、Sapporo は REST API Server として、例えばどこかの cloud instance にデプロイされている。
そのため、実行結果 (e.g., 出力ファイル, etc.) を Sapporo からダウンロードする必要がある。

そのためのスクリプトとして、`download_results.py` が用意されている。

```bash
$ python3 download_results.py -h
usage: download_results.py [-h] [endpoint] run_id [output_dir]

positional arguments:
  endpoint    The endpoint to send the request (default: http://localhost:1122)
  run_id      The RUN_ID of the executed workflow
  output_dir  The output directory to save the results (default: ./results)

optional arguments:
  -h, --help  show this help message and exit

$ python3 download_results.py http://localhost:1122 d45a404e-8d8a-43ac-bbf8-b686ee426062 ./results

$ tree results/
results/
├── outputs
│   ├── ERR034597_1.small_fastqc.html
│   ├── ERR034597_1.small.fq.trimmed.1P.fq
│   ├── ERR034597_1.small.fq.trimmed.1U.fq
│   ├── ERR034597_1.small.fq.trimmed.2P.fq
│   ├── ERR034597_1.small.fq.trimmed.2U.fq
│   └── ERR034597_2.small_fastqc.html
└── run_request.json

1 directory, 7 files
```

結果が `results` 以下にダウンロードされる。

## 3. NII-DG による RO-Crate の生成 (`package_ro_crate.py`)

前項で download した結果を、nii-dg library を用いて RO-Crate として packaging する。

```bash
$ python3 package_ro_crate.py -h
usage: package_ro_crate.py [-h] [sapporo_endpoint] [wf_results_dir]

positional arguments:
  sapporo_endpoint  The endpoint of Sapporo, which is as seen from NII-DG server
                    (default: http://sapporo-service:1122)
  wf_results_dir    The directory where the results of the workflow are stored
                    (default: ./results)

optional arguments:
  -h, --help        show this help message and exit

$ python3 package_ro_crate.py

$ ls ro-crate-metadata.json
ro-crate-metadata.json
$ head ro-crate-metadata.json
{
  "@context": "https://w3id.org/ro/crate/1.1/context",
  "@graph": [
    {
      "@id": "./",
      "@type": "Dataset",
      "hasPart": [
        {
          "@id": "outputs/ERR034597_1.small.fq.trimmed.2U.fq"
        },
```

## 4. NII-DG API Server による RO-Crate の検証

NII-DG API Server の詳細については、[api-quick-start.md](../api-quick-start.md)を参照のこと。
`0.1. Sapporo and NII-DG REST API Server の起動` より、`localhost:5000` に NII-DG API Server が起動している。

```bash
$ curl localhost:5000/healthcheck
{"message": "OK"}
```

`/validate` に生成した `ro-crate-metadata.json` を POST する。

```
$ curl -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata.json http://localhost:5000/validate
{"request_id":"bd6f18a4-1599-41d2-bcbd-75f2cda2209d"}
```

レスポンスとして、`request_id` が返ってくる。
この `request_id` を用いて、`/{request_id}` に GET し、検証結果を取得する。

```bash
$ curl localhost:5000/bd6f18a4-1599-41d2-bcbd-75f2cda2209d
{
  "request": {
    "entityIds": [],
    "roCrate": {
      "@context": "https://w3id.org/ro/crate/1.1/context",
      "@graph": [
        {
          ...
        }
      ]
    }
  },
  "requestId": "bd6f18a4-1599-41d2-bcbd-75f2cda2209d",
  "results": [],
  "status": "COMPLETE"
}
```

検証に問題がなかったため、`status` が `COMPLETE` 、`results` は空のリストになっている。

---

検証に問題がある例 (例えば、出力ファイルの名前が一致しないなど) として、`./ro-crate-metadata.json` を編集する。

```bash
$ cp ./ro-crate-metadata.json ./ro-crate-metadata_failed.json
$ sed -i 's/ERR034597_1.small_fastqc.html/ERR034597_1.small_fastqc_failed.html/' ./ro-crate-metadata_failed.json
$ diff -u ./ro-crate-metadata.json ./ro-crate-metadata_failed.json
```

編集した RO-Crate `ro-crate-metadata_failed.json` を用いて、検証を実行する。

```bash
$ curl -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata_failed.json http://localhost:5000/validate
{"request_id":"32012249-d8f7-4f64-b20d-853ea5be67b5"}

$ curl localhost:5000/32012249-d8f7-4f64-b20d-853ea5be67b5
{
  "request": {
    "entityIds": [],
    "roCrate": {
      "@context": "https://w3id.org/ro/crate/1.1/context",
      "@graph": [
        {
          ...
        }
      ]
    }
  },
  "requestId": "32012249-d8f7-4f64-b20d-853ea5be67b5",
  "results": [
    {
      "entityId": "#sapporo-run",
      "props": "sapporo.SapporoRun:outputs_ERR034597_1.small_fastqc.html",
      "reason": "The file ERR034597_1.small_fastqc.html is included in the result of re-execution, so File entity with @id ERR034597_1.small_fastqc.html is required in this crate."
    }
  ],
  "status": "FAILED"
}
```

検証に問題があったため、`status` が `FAILED` 、`results` に問題があることが示されている。

**デモ実施の準備としてNII-DG library の installが必要**
この項では、packaging 用の nii-dg library を install する。

```bash
# home directory に移動する
$ cd ../
$ ls setup.py
setup.py
$ python3 -m pip install .
...
$ python3 -m pip list | grep nii-dg
nii-dg                 1.0.0
```

**デモ実行環境への移動**

In [117]:
%cd ~/sapporo_example

/home/jovyan/sapporo_example


**execute_workflow.py を実行することで、Sapproサービスを利用してワークフローを実行
実行完了を確認して、download_results.pyを実行することで、Sapproサービス側に出力されている実行結果を取得する。**

In [118]:
%%bash
result=$(python3 execute_workflow.py http://dg01.dg.rcos.nii.ac.jp:1122)
run_id=$(echo $result|sed -e "s/'/\"/g"|jq .run_id|sed -e "s/\"//g")
echo $run_id
STATUS="RUNNING"
while [ $STATUS = "RUNNING" ]
do
    sleep 10
    RETURN=$(curl -s dg01.dg.rcos.nii.ac.jp:1122/runs/$run_id|jq .state)
    STATUS=$(echo $RETURN|sed -e "s/\"//g")
    echo $STATUS
done

python3 download_results.py http://dg01.dg.rcos.nii.ac.jp:1122 $run_id ./results
date
ls -l ./results/outputs

f29baaf4-c2fe-46c8-a03a-18399c1c0882
RUNNING
RUNNING
COMPLETE
Wed 26 Apr 2023 06:23:10 AM UTC
total 12296
-rw-r--r-- 1 jovyan users  592394 Apr 26 06:23 ERR034597_1.small_fastqc.html
-rw-r--r-- 1 jovyan users 5566650 Apr 26 06:23 ERR034597_1.small.fq.trimmed.1P.fq
-rw-r--r-- 1 jovyan users  187289 Apr 26 06:23 ERR034597_1.small.fq.trimmed.1U.fq
-rw-r--r-- 1 jovyan users 5560582 Apr 26 06:23 ERR034597_1.small.fq.trimmed.2P.fq
-rw-r--r-- 1 jovyan users   80131 Apr 26 06:23 ERR034597_1.small.fq.trimmed.2U.fq
-rw-r--r-- 1 jovyan users  592566 Apr 26 06:23 ERR034597_2.small_fastqc.html


**package_ro_crate.py を実行することで、取得して保存してあるワークフロー実行結果を使った検証を実施するためのPackagingを実施しRO-Crateで管理状況をまとめる。**

In [119]:
%%bash
python3 package_ro_crate.py
cat ro-crate-metadata.json|jq .

{
  "@context": "https://w3id.org/ro/crate/1.1/context",
  "@graph": [
    {
      "@id": "./",
      "@type": "Dataset",
      "hasPart": [
        {
          "@id": "outputs/ERR034597_1.small.fq.trimmed.1U.fq"
        },
        {
          "@id": "outputs/ERR034597_1.small.fq.trimmed.2P.fq"
        },
        {
          "@id": "outputs/ERR034597_1.small_fastqc.html"
        },
        {
          "@id": "outputs/ERR034597_1.small.fq.trimmed.1P.fq"
        },
        {
          "@id": "outputs/ERR034597_2.small_fastqc.html"
        },
        {
          "@id": "outputs/ERR034597_1.small.fq.trimmed.2U.fq"
        },
        {
          "@id": "outputs/"
        }
      ],
      "name": "example research project",
      "datePublished": "2023-04-26T06:23:11.073+00:00"
    },
    {
      "@id": "ro-crate-metadata.json",
      "@type": "CreativeWork",
      "conformsTo": {
        "@id": "https://w3id.org/ro/crate/1.1"
      },
      "about": {
        "@id": "./"
      }
    },
    

**PackagingしたRO-CrateをDG-Coreサービスに対して送付して検証を依頼する。\
statusがCOMPLETEでresultsが[]であることで検証がOKであることを知ることができる。\
検証がNGである場合はstatusがFAILEDであることで知ることができ、\
さらにNGの原因詳細は
resultsで一覧できる。**

In [120]:
%%bash
result=$(curl -s -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata.json http://dg01.dg.rcos.nii.ac.jp:5000/validate)
request_id=$(echo $result|jq .request_id|sed -e "s/\"//g")
echo $request_id

STATUS="RUNNING"
while [ $STATUS = "RUNNING" ]
do
    sleep 20 
    RETURN=$(curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .status)
    STATUS=$(echo $RETURN|sed -e "s/\"//g")
    echo $STATUS
done
curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .|tail -20

289d1a3a-8dd6-4cbc-8709-373abbef1ba9
RUNNING
COMPLETE
          "@type": "SapporoRun",
          "outputs": {
            "@id": "outputs/"
          },
          "sapporo_location": "http://sapporo-service:1122",
          "state": "COMPLETE",
          "workflow_attachment": "[{\"file_name\": \"ERR034597_2.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_2.small.fq.gz?raw=true\"}, {\"file_name\": \"ERR034597_1.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_1.small.fq.gz?raw=true\"}]",
          "workflow_engine_name": "cwltool",
          "workflow_params": "{\"fastq_1\": {\"location\": \"ERR034597_1.small.fq.gz\", \"class\": \"File\"}, \"fastq_2\": {\"location\": \"ERR034597_2.small.fq.gz\", \"class\": \"File\"}, \"nthreads\": 2}",
          "workflow_type": "CWL",
          "workflow_type_version": "v1.0",
          "workflow_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/mai

**検証失敗の例としてworkflow_attachment がwf の remote file を sapporo 上に配置する部分で、\
更には、workflow_params でその local file として配置された file を指定している。\
この「workflow_attachment の remote file の link が切れている場合」を再現するために\
workflow_attachment の remote fileが置いてあるhttps://github.com/jxta/dg-demo-2023-04\
のリポジトリ名を変更して以下を実行する。\
\
この操作を実施するために　元のexecute_workflow.py の中にインラインで記述されている\
https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/ERR034597_2.small.fq.gz　などを\
上記URLである\
\
https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_2.small.fq.gz?raw=true\
\
に変更しておく必要がある**


In [121]:
%%bash
result=$(curl -s -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata.json http://dg01.dg.rcos.nii.ac.jp:5000/validate)
request_id=$(echo $result|jq .request_id|sed -e "s/\"//g")
echo $request_id

STATUS="RUNNING"
while [ $STATUS = "RUNNING" ]
do
    sleep 20 
    RETURN=$(curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .status)
    STATUS=$(echo $RETURN|sed -e "s/\"//g")
    echo $STATUS
done
curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .|tail -20

4cec2066-9bf7-40fb-bc8a-7359b1f03d47
RUNNING
COMPLETE
          "@type": "SapporoRun",
          "outputs": {
            "@id": "outputs/"
          },
          "sapporo_location": "http://sapporo-service:1122",
          "state": "COMPLETE",
          "workflow_attachment": "[{\"file_name\": \"ERR034597_2.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_2.small.fq.gz?raw=true\"}, {\"file_name\": \"ERR034597_1.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_1.small.fq.gz?raw=true\"}]",
          "workflow_engine_name": "cwltool",
          "workflow_params": "{\"fastq_1\": {\"location\": \"ERR034597_1.small.fq.gz\", \"class\": \"File\"}, \"fastq_2\": {\"location\": \"ERR034597_2.small.fq.gz\", \"class\": \"File\"}, \"nthreads\": 2}",
          "workflow_type": "CWL",
          "workflow_type_version": "v1.0",
          "workflow_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/mai

**githubのリポジトリをPrivateに変更して再検証する。　→ FAILEDになる。**

In [126]:
%%bash
result=$(curl -s -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata.json http://dg01.dg.rcos.nii.ac.jp:5000/validate)
request_id=$(echo $result|jq .request_id|sed -e "s/\"//g")
echo $request_id

STATUS="RUNNING"
while [ $STATUS = "RUNNING" ]
do
    sleep 20 
    RETURN=$(curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .status)
    STATUS=$(echo $RETURN|sed -e "s/\"//g")
    echo $STATUS
done
curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .|tail -20

f7b50a7f-214a-4754-9608-5490dbfd76db
RUNNING
FAILED
          "workflow_attachment": "[{\"file_name\": \"ERR034597_2.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_2.small.fq.gz?raw=true\"}, {\"file_name\": \"ERR034597_1.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_1.small.fq.gz?raw=true\"}]",
          "workflow_engine_name": "cwltool",
          "workflow_params": "{\"fastq_1\": {\"location\": \"ERR034597_1.small.fq.gz\", \"class\": \"File\"}, \"fastq_2\": {\"location\": \"ERR034597_2.small.fq.gz\", \"class\": \"File\"}, \"nthreads\": 2}",
          "workflow_type": "CWL",
          "workflow_type_version": "v1.0",
          "workflow_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/main/tests/resources/cwltool/trimming_and_qc.cwl"
        }
      ]
    }
  },
  "requestId": "f7b50a7f-214a-4754-9608-5490dbfd76db",
  "results": [
    {
      "entityId": "#sapporo-run",
      "

**githubのリポジトリをPublicに変更して再検証する。　→COMPLETEになる。**

In [127]:
%%bash
result=$(curl -s -X POST -H "Content-Type: application/json" -d @./ro-crate-metadata.json http://dg01.dg.rcos.nii.ac.jp:5000/validate)
request_id=$(echo $result|jq .request_id|sed -e "s/\"//g")
echo $request_id

STATUS="RUNNING"
while [ $STATUS = "RUNNING" ]
do
    sleep 20 
    RETURN=$(curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .status)
    STATUS=$(echo $RETURN|sed -e "s/\"//g")
    echo $STATUS
done
curl -s dg01.dg.rcos.nii.ac.jp:5000/$request_id|jq .|tail -20

4c161a3d-6f64-4832-817b-4efe8ac684c9
RUNNING
COMPLETE
          "@type": "SapporoRun",
          "outputs": {
            "@id": "outputs/"
          },
          "sapporo_location": "http://sapporo-service:1122",
          "state": "COMPLETE",
          "workflow_attachment": "[{\"file_name\": \"ERR034597_2.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_2.small.fq.gz?raw=true\"}, {\"file_name\": \"ERR034597_1.small.fq.gz\", \"file_url\": \"https://github.com/jxta/dg-demo-2023-04/blob/main/ERR034597_1.small.fq.gz?raw=true\"}]",
          "workflow_engine_name": "cwltool",
          "workflow_params": "{\"fastq_1\": {\"location\": \"ERR034597_1.small.fq.gz\", \"class\": \"File\"}, \"fastq_2\": {\"location\": \"ERR034597_2.small.fq.gz\", \"class\": \"File\"}, \"nthreads\": 2}",
          "workflow_type": "CWL",
          "workflow_type_version": "v1.0",
          "workflow_url": "https://raw.githubusercontent.com/sapporo-wes/sapporo-service/mai